# Data Analysis and Feature engineering for House Price modelling

In this notebook we mix exploratory data analysis, to understand the nature of the data with feature engineering and feature selection. We make heavy use of the open source Python library [Feature-engine](https://feature-engine.readthedocs.io/en/latest/), because it contains several helpful functionalities.

[Feature-engine](https://feature-engine.readthedocs.io/en/latest/)'s transformers come along with fit and transform funcionality, to learn the necessary parameters from the data, and then transform the data accordingly. So it is very easy to learn and use. And it can also detect automatically the nature of the variables, or engineer a selected group of variables.



## Table of contents

**0. Feature Creation**

    0.1 Binary indicators
    
    0.2 Variable Aggregations    
    

**1. Categorical variables**

    1.1 Missing data

    1.2 Cardinality
    
    1.3 Rare Labels
    
    1.4 Label grouping
    
    1.5 Variable selection
    
    1.6 Encoding
    
    
**2. Time variables**

    2.1 Time elapsed calculation
    
    2.2 Missing Data
    
    
**3. Discrete variables**

    3.1 Exploratory data analysis
    
    3.2 Missing Data
    
    
**4. Numerical variables**

    4.1 Distributions
    
    4.2 Relation with house sale price
    
    4.3 Missing Data
    
    
**5. Machine learning modelling + cross validation**

In [ ]:
# let's install Feature-engine
#!pip install feature_engine

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor

from lightgbm import LGBMRegressor

# Feature-engine's modules for feature egineering
from feature_engine import creation
from feature_engine import discretisation as disc
from feature_engine import encoding as enc
from feature_engine import imputation as imp
from feature_engine import selection as sel

In [ ]:
# load training data
data = pd.read_csv('../Data/train.csv')

In [ ]:
# split training data into train and test

X_train, X_test, y_train, y_test = train_test_split(data.drop(
    ['Id', 'SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.05,
    random_state=0)

X_train.shape, X_test.shape

In [ ]:
# function to create a dataset with all the train, test and subimssion, 
# so We can easily compare the variable distributions

def create_master_data(train, test, y_train, y_test):
    
    train = train.copy()
    test = test.copy()
    
    train['target'] = y_train
    train['data'] = 'train'
    train.reset_index(drop=True, inplace=True)
    
    test['target'] = y_test
    test['data'] = 'test'
    test.reset_index(drop=True, inplace=True)
    
    master_data = pd.concat([train, test], axis=0)
    master_data.reset_index(drop=True, inplace=True)
    
    return master_data

## Target

Let's explore the target variable.

In [ ]:
# histogran to evaluate target distribution

y_train.hist(bins=50, density=True)
y_test.hist(bins=50, density=True)

plt.legend(["Train", "Test"])
plt.ylabel('Number of houses')
plt.xlabel('Sale Price')
plt.show()

As expected, the 2 distributions are very similar, because the datasets come from a random division of the original data.

In [ ]:
# let's transform the target, we are optimising for the log 
# of the target as per competition rules

np.log(y_train).hist(bins=50, density=True)
np.log(y_test).hist(bins=50, density=True)

plt.ylabel('Number of houses')
plt.xlabel('Log of Sale Price')
plt.show()

In [ ]:
# density plot instead of histogram
np.log(y_train).plot.density()
np.log(y_test).plot.density()

plt.xlabel('Log of Sale Price')
plt.show()

In [ ]:
# let's transform the target with the log

y_train = np.log(y_train)

y_test = np.log(y_test)

## Feature Creation

In this section, We create new variables, from those already in the dataset.

In [ ]:
# if the house has a pool
X_train['HasPool'] = np.where(X_train['PoolArea']>0,1,0)
X_test['HasPool'] = np.where(X_test['PoolArea']>0,1,0)


# if the house has an Alley
X_train['HasAlley'] = np.where(X_train['Alley'].isnull(), 0, 1)
X_test['HasAlley'] = np.where(X_test['Alley'].isnull(), 0, 1)


In [ ]:
# create function to plot the median sale price vs a list of variables

def plot_median_price(variables, limits):
    
    tmp = pd.concat([X_train[variables].reset_index(drop=True),
                 y_train.reset_index(drop=True)], axis=1)

    # plot mean sale price per null value or otherwise
    g = sns.PairGrid(tmp, x_vars=variables, y_vars=['SalePrice'])
    g.map(sns.barplot)
    plt.ylim(limits)   
    plt.show()

In [ ]:
# plot the mean house sale price for the created variables
# to see if they add some value

plot_median_price(['HasAlley', 'HasPool'], (1.5,3.5))

Not clear if these variables add a lot of value. The difference in price is small, but then the price is in log scale.

In [ ]:
# The variables Condition1 and Condition2 indicate Proximity to various conditions. 

# Create a variable that aggregates both to indicate if the house is
# close to more than 1 ammenity. If the house is close to more than 1
# ammenity, the conditions take different values.

X_train['Condition_total'] = np.where(X_train['Condition2'] == X_train['Condition1'], 0, 1)
X_test['Condition_total'] = np.where(X_test['Condition2'] == X_test['Condition1'], 0, 1)

# inspect the variable
X_train['Condition_total'].value_counts()

Roughly 13% of the houses are proximal to 2 different ammenities.

In [ ]:
# Exterior 1s and 2nd indicate the Exterior material covering on house:
# Create a variable that aggregates both, like we did for contition

X_train['Exterior_total'] = np.where(X_train['Exterior1st'] == X_train['Exterior2nd'], 0, 1)
X_test['Exterior_total'] = np.where(X_test['Exterior1st'] == X_test['Exterior2nd'], 0, 1)

# inspect the variable
X_train['Exterior_total'].value_counts()

Roughly 15% of the houses ashow 2 materials in the exterior.

In [ ]:
# plot the mean house sale price for the created variables

plot_median_price(['Condition_total', 'Exterior_total'], (1.5,3.5))

Again, it is unclear if these new variables add a lot of value. The difference in price seems minimal.

## Categorical Variables

In [ ]:
# let's identify the categorical variables

categorical = [var for var in data.columns if data[var].dtype == 'O']

# MSSubClass is also categorical by definition, despite its numeric values
categorical = categorical + ['MSSubClass']

# number of categorical variables
len(categorical)

In [ ]:
# cast all variables as categorical

X_train[categorical] = X_train[categorical].astype('O')
X_test[categorical] = X_test[categorical].astype('O')

### Missing Data

In [ ]:
# which categorical variables have missing data?

# capture categorical variables with NA in a dictionary
null_cat = {var: data[var].isnull().mean() for var in categorical if data[var].isnull().mean()>0}

# plot
pd.Series(null_cat).sort_values().plot.bar(figsize=(10,4))
plt.ylabel('Percentage of missing data')
plt.axhline(y = 0.90, color = 'r', linestyle = '-') 
plt.axhline(y = 0.80, color = 'g', linestyle = '-') 

plt.show()

There are a few variables with a lot of data missing, and a few variables with few missing data. In particular, there are 3 variables for which > 90% of the values are missing (above red line)

Let's examine if the variables with NA are somewhat informative of the house sale price.

In [ ]:
# create a temporary dataset with the interest variables

tmp = pd.concat([X_train[['Alley', 'MiscFeature', 'PoolQC']].reset_index(drop=True),
                 y_train.reset_index(drop=True)], axis=1)

# replace null values by 1, or 0 otherwise
for var in ['Alley', 'MiscFeature', 'PoolQC']:
    tmp[var] = np.where(tmp[var].isnull(),1,0)


# plot mean sale price per null value or otherwise
g = sns.PairGrid(tmp, x_vars=['Alley', 'MiscFeature', 'PoolQC'], y_vars=['SalePrice'])
g.map(sns.barplot)
plt.ylim(1,5)   
plt.show()

They don't seem to be hughly predictive so we will drop them (we already captured 2 of them in a binary feature anyways, at the beginning of the notebook).

[DropFeatures](https://feature-engine.readthedocs.io/en/latest/selection/DropFeatures.html)

In [ ]:
# DropFeatures allows me to drop selected feature groups from data

drop_features = sel.DropFeatures(features_to_drop = ['Alley', 'MiscFeature', 'PoolQC'])

X_train = drop_features.fit_transform(X_train)
X_test = drop_features.transform(X_test)

X_train.shape, X_test.shape

Let's impute the missing data in categorical variables.

[CategoricalImputer](https://feature-engine.readthedocs.io/en/latest/imputation/CategoricalImputer.html)

In [ ]:
# impute missing data, categorical variables are detected automatically
# the imputer replaces missing data with the string 'Missing'

cat_imputer = imp.CategoricalImputer(return_object=True)

cat_imputer.fit(X_train)

# the variables to impute are stored in the variables attribute
cat_imputer.variables[0:10]

In [ ]:
# the number of categorical variables detected by Feature-engine

len(cat_imputer.variables)

In [ ]:
# the imputer will add a string 'Missing' to each categorical variable

cat_imputer.imputer_dict_

In [ ]:
# remove missing data

X_train = cat_imputer.transform(X_train)
X_test = cat_imputer.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# check that we do not have more missing data in categorical variables
# if we do, the list should not be empty

[c for c in cat_imputer.variables if X_train[c].isnull().sum()>0]

## Quasi-constant variables

Let's inspect if there are some variables that show predominantly 1 value  in all observations. We can find them automatically with the folliwing class from Feature-engine:

[DropConstantFeatures](https://feature-engine.readthedocs.io/en/latest/selection/DropConstantFeatures.html)

In [ ]:
# assign the categorical variable list to the categorical variable name

categorical = cat_imputer.variables

# we still have 41 categorical variables in the dataset
len(categorical)

In [ ]:
# We ask the transformer to remove all variables that show the same value in more than
# 94% of the observations (tol=0.94)
constant = sel.DropConstantFeatures(tol=0.94, variables=categorical)

# find constant features
constant.fit(X_train)

# the quasi-constant features are stored in this attribute
constant.features_to_drop_

In [ ]:
# put data together for analysis

data = create_master_data(X_train, X_test, y_train, y_test)

data.shape

In [ ]:
# plot number of observations per category, per variable
# so that we see that these variables show mostly 1 value in most of the 
# observations

for variable in constant.features_to_drop_:
    
    data.groupby(variable)['data'].value_counts().unstack().plot.bar(figsize=(10,5))
    plt.title(variable)
    plt.ylabel('Number of houses')
    plt.show()

We see that in most of these variables, most observations show the same value. We will re-map these variables to group all the categories that are less frequent into a new category called 'Other'.

In [ ]:
# I will re-group these variables into either the majoritarian category
# or "Other"

# if a category is present in less than 10% of the observations, we group it with other
# infrequent categories (tol param)

rare_enc = enc.RareLabelEncoder(tol = 0.1,
                                n_categories=1, # number of minimum categories per variable for the grouping to proceed
                                variables=constant.features_to_drop_, # the variables to pre-process
                                replace_with='Other', # the label to use to replace the original category
                               )

rare_enc.fit(X_train)

In [ ]:
# make the grouping

X_train = rare_enc.transform(X_train)
X_test = rare_enc.transform(X_test)

In [ ]:
# Now, when we plot the data again, we should see only 2 categories in each variable

# put data together for analysis
data = create_master_data(X_train, X_test, y_train, y_test)

for variable in constant.features_to_drop_:
    
    data.groupby(variable)['data'].value_counts().unstack().plot.bar(figsize=(10,5))
    plt.title(variable)
    plt.ylabel('Number of houses')
    plt.show()

## Quality variables

There are a number of variables that refer to the quality of some aspect of the house, for example the garage, or the fence, or the kitchen. I will replace these categories by numbers increasing with the quality of the place or room.

- Ex = Excellent
- Gd = Good
- TA = Average/Typical
- Fa =	Fair
- Po = Poor

In [ ]:
qual_mappings = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'Missing': 0, 'NA': 0}

qual_vars = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
             'HeatingQC', 'KitchenQual', 'FireplaceQu',
             'GarageQual', 'GarageCond',
            ]

for var in qual_vars:
    X_train[var] = X_train[var].map(qual_mappings)
    X_test[var] = X_test[var].map(qual_mappings)

In [ ]:
exposure_mappings = {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4, 'Missing': 0, 'NA': 0}

var = 'BsmtExposure'

X_train[var] = X_train[var].map(exposure_mappings)
X_test[var] = X_test[var].map(exposure_mappings)

In [ ]:
finish_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}

finish_vars = ['BsmtFinType1', 'BsmtFinType2']

for var in finish_vars:
    X_train[var] = X_train[var].map(finish_mappings)
    X_test[var] = X_test[var].map(finish_mappings)

In [ ]:
garage_mappings = {'Missing': 0, 'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}

var = 'GarageFinish'

X_train[var] = X_train[var].map(garage_mappings)
X_test[var] = X_test[var].map(garage_mappings)

In [ ]:
fence_mappings = {'Missing': 0, 'NA': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}

var = 'Fence'

X_train[var] = X_train[var].map(fence_mappings)
X_test[var] = X_test[var].map(fence_mappings)

In [ ]:
# capture all quality variables

qual_vars  = qual_vars + finish_vars + ['BsmtExposure','GarageFinish','Fence']

In [ ]:
# now let's plot the house mean sale price based on the quality of the 
# various attributes

# put data together for analysis
data = create_master_data(X_train, X_test, y_train, y_test)

for var in qual_vars:
    # make boxplot with Catplot
    sns.catplot(x=var, y='target', data=data[data['data']=='train'], kind="box", height=4, aspect=1.5)
    # add data points to boxplot with stripplot
    sns.stripplot(x=var, y='target', data=data[data['data']=='train'], jitter=0.1, alpha=0.3, color='k')
    plt.show()
    

For many quality variables, we see an increase in the house price, with the quality.

### Cardinality

Cardinality indicates the number of unique values or categories per variable. The highest the cardinality, the more difficult the variable is to handle. Although it could provide more information.

In [ ]:
# let's collect the remaining categorical variables

categorical = [c for c in categorical if c not in constant.features_to_drop_]

categorical = [c for c in categorical if c not in qual_vars]

len(categorical)

In [ ]:
# with nunique from pandas we count the number of unique categories per variable

data[categorical].nunique().sort_values().plot.bar(figsize=(15,5))
plt.ylabel('Number of categories per variable')
plt.show()

Some variables have few categories (low cardinality), others have a lot of categories (high cardinality). 

In addition to the number of unique categories, it is important to know how many observations in the dataset show each category. Rare categories tend to bring problems when building ML models, with little reliable information, precisely, because there are not a lot of observations to learn from.

In [ ]:
# plot number of observations / houses per category, per variable

for variable in categorical:
    data.groupby(variable)['data'].value_counts().unstack().plot.bar(figsize=(10,5))
    plt.title(variable)
    plt.ylabel('Number of houses')
    plt.show()

We see that pretty much for every variable, some categories are shared by many houses, and some are shown only by very few houses. Then, we wouldn't know if we should trust the Sale Price of houses with these few categories, because we have few houses to learn from.

Another problem that comes up with rare categories, is that they may appear only in the train set or only in the test set. So if it appears only in the train set, they may cause over-fitting. But if it appears only on the test set, then the model does not know what to make of that category, and it may crash.

**We will group infrequent labels together into one umbrella category called 'Rare'**. But before that, let's have a look at the mean sale price per category, to see if we find some value in any of these variables that we can capitalize.

In [ ]:
# the following function calculates:

# 1) the percentage of houses per category
# 2) the mean SalePrice per category

# and returns a dataframe with those 2 variables

def calculate_mean_target_per_category(df, var):
    
    df = pd.concat([df, y_train], axis=1)
    
    # total number of houses
    total_houses = len(df)

    # percentage of houses per category
    temp_df = pd.Series(df[var].value_counts() / total_houses).reset_index()
    temp_df.columns = [var, 'perc_houses']

    # add the mean SalePrice
    temp_df = temp_df.merge(df.groupby([var])['SalePrice'].mean().reset_index(),
                            on=var,
                            how='left')

    return temp_df

In [ ]:
# now we use the function for the variable 'Neighborhood'
temp_df = calculate_mean_target_per_category(X_train, 'Neighborhood')

temp_df.head()

In [ ]:
# Now we create a function to plot the
# category frequency and mean SalePrice.

# This will help us visualise the relationship between the
# target and the labels of the categorical variable

def plot_categories(df, var):
    
    fig, ax = plt.subplots(figsize=(8, 4))
    plt.xticks(df.index, df[var], rotation=90)

    ax2 = ax.twinx()
    ax.bar(df.index, df["perc_houses"], color='lightgrey')
    ax2.plot(df.index, df["SalePrice"], color='green', label='Seconds')
    ax.axhline(y=0.05, color='red')
    ax.set_ylabel('percentage of houses per category')
    ax.set_xlabel(var)
    ax2.set_ylabel('Average Sale Price per category')
    plt.show()

In [ ]:
# plot house count and sale price for Neighbourhood.

plot_categories(temp_df, 'Neighborhood')

From the above plot, we can see that there are expensive neighbourhoods and cheap neighbourhoods.

In [ ]:
temp_df['SalePrice'].describe()

In [ ]:
# make a list of the least expensive neighbourhoods

cheap_neighbourhoods = temp_df[temp_df['SalePrice']<2.47]['Neighborhood'].unique()

cheap_neighbourhoods

In [ ]:
# make a list of the most expensive neighbourhoods

expensive_neighbourhoods = temp_df[temp_df['SalePrice']>=2.47]['Neighborhood'].unique()

expensive_neighbourhoods

In [ ]:
# new variable that segregates neigbourhoods as per the previous lists

X_train['Neigh_Price'] = np.where(X_train['Neighborhood'].isin(cheap_neighbourhoods),
                                  0, np.where(X_train['Neighborhood'].isin(expensive_neighbourhoods),
                                              2, 1))

X_test['Neigh_Price'] = np.where(X_test['Neighborhood'].isin(cheap_neighbourhoods),
                                  0, np.where(X_test['Neighborhood'].isin(expensive_neighbourhoods),
                                              2, 1))

# let's inspect the new variable
X_train['Neigh_Price'].value_counts()

In [ ]:
# let's inspect the value of our new variable

# put data together for analysis
data = create_master_data(X_train, X_test, y_train, y_test)

sns.catplot(x='Neigh_Price', y='target', data=data[data['data']=='train'], kind="box", height=4, aspect=1.5)
# add data points to boxplot with stripplot
sns.stripplot(x='Neigh_Price', y='target', data=data[data['data']=='train'], jitter=0.1, alpha=0.3, color='k')
plt.show()

Looks good!

In [ ]:
# let's plot for the remaining categorical variables
# the count of houses per category and the mean sale price

for col in categorical:
    
    # we plotted this variable already
    if col !='Neighborhood':
        
        # re using the functions I created
        temp_df = calculate_mean_target_per_category(data, col)
        plot_categories(temp_df, col)

The other variables, don't show a clear increase in house for certain categories. So we will not derive new features from them.

In [ ]:
# Now we will group infrequent labels together:

# if a category is present in less than 5% of the observations, we group it with other
# infrequent categories (tol param)

rare_enc = enc.RareLabelEncoder(tol = 0.05,
                                n_categories=4, # number of minimum categories per variable for the grouping to proceed
                                variables=categorical
                               )

rare_enc.fit(X_train)

The warnings tell us that there are a few categorical variables that we indicated to the variables parameter, that have less than 4 unique categories. So the transformer will not pre-process those.

In [ ]:
X_train = rare_enc.transform(X_train)
X_test = rare_enc.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# put data together for analysis

data = create_master_data(X_train, X_test, y_train, y_test)

In [ ]:
# now we can plot the variables with grouped categories

for variable in categorical:
    data.groupby(variable)['data'].value_counts().unstack().plot.bar(figsize=(10,5))
    plt.title(variable)
    plt.ylabel('Number of houses')
    plt.show()

We see that after grouping the variables, the categories appear in all the two datasets, train and test. And also, they are shared by at least more than 5% of the observations in the datasets.

### Categorical variable importance

Let's plot the House Sale Price distribution per category per variable to understand if there is a difference.

**Boxplot**: indicates the median value of the house, and the interquartile range distance, which contains most of the houses. The rombos above and below are outliers for that distribution.

**Jitter**: on top of the box plot we plot the houses individually as dots, this gives us an idea of how many houses show that category. More dots, more houses.

If the box plots are at the same height, then the categories or the variable do not show predictive power. But if the show different heights for the different categories, then they might.

In [ ]:
for variable in categorical:
    # make boxplot with Catplot
    sns.catplot(x=variable, y='target', data=data[data['data']=='train'], kind="box", height=4, aspect=1.5)
    # add data points to boxplot with stripplot
    sns.stripplot(x=variable, y='target', data=data[data['data']=='train'], jitter=0.1, alpha=0.3, color='k')
    plt.show()

Some categories seem to correlate with higher or lower house sale prices. And some, not at all. See for example the plot for the variable **Foundation**. The different foundations seem to correlate with lower sale prices. The same is true for the variable **SaleCondition** and others.

We can try and determine which of the variables are useful using an approach used in the KDD2009 data science competition. The approach consists in replacing the category by the mean of the target. And then using that replacement as a prediction, and evaluate the performance comparing the prediction to the real value of the house price.

We can perform all of this, very easily with a class from Feature-engine.

Check [SelectByTargetMeanPerformance](https://feature-engine.readthedocs.io/en/latest/selection/SelectByTargetMeanPerformance.html) for more details.

### Encode categorical variables

To use categorical variables in machine learning models, we need to encode them into numbers. At least for the Gradient Boosting Classifier from Sklearn.

We are going to assign numbers to the variables, but these numbers will be assigned from smaller to bigger, based on the mean sale price per category. This way, we create (hopefully) a monotonic relationship between the encoded variable and the target, which may boost the performance of the model.

[OrdinalEncoder](https://feature-engine.readthedocs.io/en/latest/encoding/OrdinalEncoder.html)

In [ ]:
# let's identify all remaining categorical variables (remember that we encoded
# some into numbers already)

categorical = [var for var in X_train.columns if X_train[var].dtype == 'O']

# MSSubClass is also categorical by definition, despite its numeric values
categorical = categorical + ['MSSubClass']

# number of categorical variables
len(categorical)


In [ ]:
# cast all variables as categorical

#X_train[categorical] = X_train[categorical].astype('O')
#X_test[categorical] = X_test[categorical].astype('O')

In [ ]:
# set up the ordinal encoder from Feature-engine

encoder = enc.OrdinalEncoder(encoding_method='ordered',
                             variables=categorical)

encoder.fit(X_train, y_train)

In [ ]:
# in this attribute we find the numbers that will replace each category in each variable

encoder.encoder_dict_

In [ ]:
# encode the variables

X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# Now let's examine the monotonic relationships

for var in categorical:
    plt.scatter(X_train[var], y_train, alpha=0.2)
    plt.ylabel('Log of Sale Price')
    plt.xlabel(var)
    plt.show()

For some variables, we see an increase in the house price with the encoded variable value. 

In [ ]:
# check that NAN were not introduced during the encoding

[c for c in categorical if X_train[c].isnull().sum()>0]

In [ ]:
# check that NAN were not introduced during the encoding

[c for c in categorical if X_test[c].isnull().sum()>0]

## Time variables

There are a few variables that show time, for example the year in which the house was sold, or the garage was built.

In [ ]:
year_vars = [var for var in data.columns if 'Yr' in var or 'Year' in var]

year_vars

Let's plot the sale price over the years.

In [ ]:
data = create_master_data(X_train, X_test, y_train, y_test)

In [ ]:
# make boxplot with Catplot
sns.catplot(x='YrSold', y='target', data=data[data['data']=='train'], kind="box", height=4, aspect=1.5)
# add data points to boxplot with stripplot
sns.stripplot(x='YrSold', y='target', data=data[data['data']=='train'], jitter=0.1, alpha=0.3, color='k')
plt.show()

We see that the sale price seems stable over the years for which we have data.

Now let's plot the house price vs the last time where something was remodelled in the house. I expect more expensive houses if they were recently remodelled.

In [ ]:
for variable in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    plt.scatter(X_train[variable], y_train)
    plt.ylabel('Log of sale Price')
    plt.xlabel(variable)
    plt.show()

There seems to be a slight linear trend to increase in house prices, for those houses that were remodelled in later years.

Let's capture the difference between the year in which something was remodelled or built and the sale time.

[CombineWithFeatureReference](https://feature-engine.readthedocs.io/en/latest/creation/CombineWithReferenceFeature.html)

In [ ]:
# this transformer will substract all the variables in the reference list, from YrSold
# to determine the age of remodelling at point of sale

create = creation.CombineWithReferenceFeature(
    variables_to_combine = ['YrSold'],
    reference_variables = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'],
    operations=['sub'],
)

create.fit(X_train)

In [ ]:
X_train = create.transform(X_train)
X_test = create.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# we can see the new variables at the right of the dataframe
X_train.head()

In [ ]:
# if the variable 'YearRemodAdd' shows the same value as the
# variable 'YearBuilt', that means that the house has not been remodelled

# so let's create a new feature that captures this.

remodelled = creation.CombineWithReferenceFeature(
    variables_to_combine = ['YearRemodAdd'],
    reference_variables = ['YearBuilt'],
    operations=['sub'],
)

remodelled.fit(X_train)

In [ ]:
X_train = remodelled.transform(X_train)
X_test = remodelled.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# we can see the new variable at the right of the dataframe
X_train.head()

In [ ]:
# now we need to remove the original time features

drop_features = sel.DropFeatures(features_to_drop =['YearBuilt', 'YearRemodAdd', 'GarageYrBlt','YrSold'])

X_train = drop_features.fit_transform(X_train)
X_test = drop_features.transform(X_test)

X_train.shape, X_test.shape

### Missing data

In [ ]:
# let's check if the variables have missing data

X_train[[
    'YrSold_sub_YearBuilt', 'YrSold_sub_YearRemodAdd' ,'YrSold_sub_GarageYrBlt', 'YearRemodAdd_sub_YearBuilt']
].isnull().mean()

In [ ]:
X_test[[
    'YrSold_sub_YearBuilt', 'YrSold_sub_YearRemodAdd' ,'YrSold_sub_GarageYrBlt', 'YearRemodAdd_sub_YearBuilt']
].isnull().mean()

YrSold_sub_GarageYrBlt shows missing data. We will impute this altogether when we impute numerical variables later on in the notebook.

The final temporal variable is the month in which the house was sold. Let's check if there is some price change depending on the month of the sale.

In [ ]:
# make boxplot with Catplot
sns.catplot(x='MoSold', y='target', data=data[data['data']=='train'], kind="box", height=4, aspect=1.5)
# add data points to boxplot with stripplot
sns.stripplot(x='MoSold', y='target', data=data[data['data']=='train'], jitter=0.1, alpha=0.3, color='k')
plt.show()

The mean house sale price seems similar throughout the months, but there is a difference in the number of houses sold, with less houses sold in Jan and Feb, and more houses sold towards the summer months of the northern hemisphere.

## Discrete variables

In [ ]:
discrete = [
    var for var in X_train.columns if X_train[var].dtype != 'O'
    and var not in year_vars
    and var not in qual_vars+categorical
    and len(X_train[var].unique()) < 20 
]

# number of discrete variables
len(discrete)

Let's see if there is a relationship between the values of discrete houses and the mean sale price.

In [ ]:
for var in discrete:
    # make boxplot with Catplot
    sns.catplot(x=var, y='target', data=data[data['data']=='train'], kind="box", height=4, aspect=1.5)
    # add data points to boxplot with stripplot
    sns.stripplot(x=var, y='target', data=data[data['data']=='train'], jitter=0.1, alpha=0.3, color='k')
    plt.show()

For most of these variables, there is a correlation between the number assigned to the house and the sale price. Particularly for those variables which values are determined by people, for example OverallQual, or OverallCond. But also for variables like number of bathrooms or rooms.

The only variable that seems to have only 1 value predominantly is PoolArea, so we will remove it from the dataset.

In [ ]:
# remove original feature

drop_features = sel.DropFeatures(features_to_drop =['PoolArea'])

X_train = drop_features.fit_transform(X_train)
X_test = drop_features.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
### Missing data

# let's capture the discrete variables with missing data

null_disc = {var: data[var].isnull().mean() for var in discrete if data[var].isnull().mean()>0}

# plot
if len(null_disc) > 0:
    pd.Series(null_disc).sort_values().plot.bar(figsize=(10,4))
    plt.ylabel('Percentage of missing data')
    plt.show()

If there discrete variables with NA, we will impute these altogether with numerical variables 

In [ ]:
# get variable names

pd.Series(null_disc).sort_values().index

## Numerical variables

In [ ]:
numerical = [
    var for var in X_train.columns 
    if var not in categorical + qual_vars + discrete + year_vars
]

len(numerical)

In [ ]:
# let's examine the distribution of the numerical continuous variables

X_train[numerical].hist(bins=50, figsize=(15,15))

plt.show()

There are some variables that show predominantly 1 value, like MiscVal, ScreenPorch, LowQualFinSF. We could remove them with the constant features selector. But we could also let the model decide if they are important or not. So we will do this, this time.

In [ ]:
# let's plot the sale price vs the numerical variables

tmp = pd.concat([X_train, y_train], axis=1)

sns.pairplot(data=tmp,
             y_vars='SalePrice',
             x_vars=['LotFrontage',
                     'LotArea',
                     'MasVnrArea',
                     'BsmtFinSF1',
                     'BsmtFinSF2' ]) 
plt.show()

We see that the higher the value of the variable, the higher the sale price, for most of these variables.

In [ ]:
sns.pairplot(data=tmp,
             y_vars=['SalePrice'],
             x_vars=['BsmtUnfSF',
                     'TotalBsmtSF',
                     '1stFlrSF',
                     '2ndFlrSF',
                     'LowQualFinSF' ])
plt.show()

Same here, the higher the value of the variable, the higher the sale price, in general.

In [ ]:
sns.pairplot(data=tmp,
             y_vars=['SalePrice'],
             x_vars=['GrLivArea',
                     'GarageArea',
                     'WoodDeckSF',
                     'OpenPorchSF',
                     'EnclosedPorch' ])

plt.show()

In [ ]:
sns.pairplot(data=tmp,
             y_vars=['SalePrice'],
             x_vars=['3SsnPorch',
                     'ScreenPorch',
                     'MiscVal'])

plt.show()

For these variables, there does not seem to be a clear tendency.

In [ ]:
# and now the time variables we created before

sns.pairplot(data=tmp,
             y_vars=['SalePrice'],
             x_vars=['YrSold_sub_YearBuilt', 
                     'YrSold_sub_YearRemodAdd' ,
                     'YrSold_sub_GarageYrBlt',
                     'YearRemodAdd_sub_YearBuilt'])

plt.show()

There is some promise in these variables

### Missing Data

In [ ]:
# concatenate data sources
data = create_master_data(X_train, X_test, y_train, y_test)

# capture numerical variables with NA
null_num = {var: data[var].isnull().mean() for var in numerical if data[var].isnull().mean()>0}

# plot
pd.Series(null_num).sort_values().plot.bar(figsize=(5,4))
plt.ylabel('Percentage of missing data')
plt.show()

There are a few variables that show missing data.

Let's add a missing indicator first and then impute the missing data with the median value of the variable. 

[MeanMedianImputer](https://feature-engine.readthedocs.io/en/latest/imputation/MeanMedianImputer.html)

[AddMissingIndicator](https://feature-engine.readthedocs.io/en/1.0.x/imputation/AddMissingIndicator.html)

In [ ]:
vars_to_impute = [c for c in null_num.keys()]

# add the discrete variables with NA
vars_to_impute = vars_to_impute + ['GarageCars', 'BsmtFullBath', 'BsmtHalfBath']

indicator = imp.AddMissingIndicator(missing_only = False,
                                variables = vars_to_impute,
                               )

indicator.fit(X_train)

In [ ]:
# in this attribute we find the variables for which
# the missing indicators will be added

indicator.variables_

In [ ]:
X_train = indicator.transform(X_train)
X_test = indicator.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# now replace NA by the median value

num_imputer = imp.MeanMedianImputer(imputation_method = 'median',
                                    variables = vars_to_impute,
                                   )

num_imputer.fit(X_train)

# the median for imputation for each numerical variable
num_imputer.imputer_dict_

In [ ]:
X_train = num_imputer.transform(X_train)
X_test = num_imputer.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
[c for c in X_train.columns if X_train[c].isnull().sum()>0]

In [ ]:
[c for c in X_test.columns if X_test[c].isnull().sum()>0]

In [ ]:
# create some more features

# total number of basement bathrooms
bath_bsmt = creation.MathematicalCombination(
    variables_to_combine=['BsmtHalfBath', 'BsmtFullBath'],
    math_operations=['sum'],
    new_variables_names=['BsmtBath_total'],
)

# total number of bathrooms
bath_ground = creation.MathematicalCombination(
    variables_to_combine=['FullBath', 'HalfBath'],
    math_operations=['sum'],
    new_variables_names=['Bath_total'],
)

X_train = bath_bsmt.fit_transform(X_train)
X_test = bath_bsmt.transform(X_test)

X_train = bath_ground.fit_transform(X_train)
X_test = bath_ground.transform(X_test)

X_train.shape, X_test.shape

## Model with nested cross-validation
Nice explanation here:
https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

In [ ]:
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [ ]:
def nested_cross_val(model, grid):
    
    # configure the cross-validation procedure
    cv_outer = KFold(n_splits=5, shuffle=True, random_state=1)

    # enumerate splits
    outer_results = list()

    for train_ix, test_ix in cv_outer.split(X_train):

        # split data
        xtrain, xtest = X_train.loc[train_ix, :], X_train.loc[test_ix, :]
        ytrain, ytest = y_train[train_ix], y_train[test_ix]

        # configure the cross-validation procedure
        cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)

        # define search
        search = GridSearchCV(model, grid, scoring='neg_root_mean_squared_error', cv=cv_inner, refit=True)

        # execute search
        search.fit(xtrain, ytrain)

        # evaluate model on the hold out dataset
        yhat = search.predict(xtest)

        # evaluate the model
        rmse = mean_squared_error(ytest, yhat, squared=False)

        # store the result
        outer_results.append(rmse)

        # report progress
        print('>rmse_outer=%.3f, rmse_inner=%.3f, cfg=%s' % (rmse, search.best_score_, search.best_params_))

    # summarize the estimated performance of the model
    print('rmse_outer: %.3f +- %.3f' % (np.mean(outer_results), np.std(outer_results)))
    
    return search.fit(X_train, y_train)

### GBM
Takes a long time!
Six minutes on my laptop!

In [ ]:
%%time
gbm = GradientBoostingRegressor(
    loss='ls',
    n_estimators=100,
    criterion='friedman_mse',
    min_samples_split=2,
    max_depth=3,
    random_state=0,
    n_iter_no_change=2,
    tol=0.0001,
    )

param_grid = dict(
    loss=['ls', 'huber'],
    n_estimators=[10, 20, 50, 100, 200, 500, 1000, 2000],
    min_samples_split=[0.1, 0.3, 0.5],
    max_depth=[1,2,3,4,None],
    )

search = nested_cross_val(gbm, param_grid)

In [ ]:
results = pd.DataFrame(search.cv_results_)

results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['params','mean_test_score', 'std_test_score']]

results['mean_test_score'].plot(yerr=[results['std_test_score'], results['std_test_score']], subplots=True)

plt.ylabel('Mean test score')

In [ ]:
#results

In [ ]:
# let's get the predictions

X_train_preds = search.predict(X_train)
X_test_preds = search.predict(X_test)


print('Train rmse: ', mean_squared_error(y_train, X_train_preds, squared=False))
print('Test rmse: ', mean_squared_error(y_test, X_test_preds, squared=False))
print()
print('Train r2: ', r2_score(y_train, X_train_preds))
print('Test r2: ', r2_score(y_test, X_test_preds))

In [ ]:
# Remember house price has been logged so we should exponentiate!
# np.exp()

### Light GBM
Takes really long!

In [ ]:
%%time
# Light GBM

lgbm_param = {
    "num_leaves": [6, 8, 20, 30],
    "max_depth": [2, 4, 6, 8, 10],
    "n_estimators": [50, 100, 200, 500],
    'colsample_bytree': [0.3, 1.0],
}

lgbm = LGBMRegressor(
    learning_rate = 0.1, 
    min_child_weight = 0.4,
    objective='regression', 
    random_state=0)

search = nested_cross_val(lgbm, lgbm_param)

In [ ]:
results = pd.DataFrame(search.cv_results_)

results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['params','mean_test_score', 'std_test_score']]

results['mean_test_score'].plot(yerr=[results['std_test_score'], results['std_test_score']], subplots=True)

plt.ylabel('Mean test score')

In [ ]:
# let's get the predictions

X_train_preds = search.predict(X_train)
X_test_preds = search.predict(X_test)


print('Train rmse: ', mean_squared_error(y_train, X_train_preds, squared=False))
print('Test rmse: ', mean_squared_error(y_test, X_test_preds, squared=False))
print()
print('Train r2: ', r2_score(y_train, X_train_preds))
print('Test r2: ', r2_score(y_test, X_test_preds))

In [ ]:
# Remember house price has been logged so we should exponentiate!
# np.exp()

## Feature selection

In [ ]:
sel_perf = sel.SelectBySingleFeaturePerformance(
    estimator=DecisionTreeRegressor(random_state=2, max_depth=2),
    scoring='r2',
    cv=3,
    threshold=0.01,
    variables=None,
)

sel_perf.fit(X_train, y_train)

In [ ]:
# in the attribute feature_performance_ the class stores the performance
# of each feature

pd.Series(sel_perf.feature_performance_).sort_values().plot.bar(figsize=(15,5))
plt.ylabel('r2')

# the red line is the threshold we selected.
plt.axhline(y = pd.Series(sel_perf.feature_performance_).mean(),
            color = 'g', linestyle = '-')

# the red line is the threshold we selected.
plt.axhline(y = pd.Series(sel_perf.threshold).mean(),
            color = 'r', linestyle = '-') 

plt.show()

In [ ]:
# number of features to drop

print('total features: ', X_train.shape[1])
print('features to drop: ', len(sel_perf.features_to_drop_))
print('remaining features: ', X_train.shape[1] - len(sel_perf.features_to_drop_))

In [ ]:
%%time
# let's try different feature selection method

rfm = RandomForestRegressor(
    n_estimators=100,
    random_state=0,
    max_depth=2,
    )

rfe = sel.RecursiveFeatureElimination(
    estimator = rfm,
    scoring ='neg_root_mean_squared_error',
    cv=3,
    threshold=0.0001,
    variables=None,
)

rfe.fit(X_train, y_train)

In [ ]:
# performance of the gbm built using all features

rfe.initial_model_performance_

In [ ]:
# in the attribute performance_drifts_ the class stores the performance
# of each feature the drop in performance when the feature was removed

pd.Series(rfe.performance_drifts_).sort_values().plot.bar(figsize=(15,5))
plt.ylabel('neg rmse')

# the red line is the threshold we selected.
plt.axhline(y = pd.Series(rfe.performance_drifts_).mean(),
            color = 'g', linestyle = '-')

plt.axhline(y = rfe.threshold,
            color = 'r', linestyle = '-') 

plt.show()

In [ ]:
# in the attribute feature_importances_ the class stores the importance
# of each feature, derived from the gbm

pd.Series(rfe.feature_importances_).sort_values().plot.bar(figsize=(15,5))
plt.ylabel('neg rmse')

# the red line is the threshold we selected.
plt.axhline(y = pd.Series(rfe.feature_importances_).mean(),
            color = 'r', linestyle = '-') 
plt.show()

In [ ]:
# number of features to drop

print('total features: ', X_train.shape[1])
print('features to drop: ', len(rfe.features_to_drop_))
print('remaining features: ', X_train.shape[1] - len(rfe.features_to_drop_))

In [ ]:
X_train = sel_perf.transform(X_train) 
X_test = sel_perf.transform(X_test) 

X_train.shape, X_test.shape

## Train model with cross-validation, searching for best parameters

In the rest of the notebook, I will perform a Random Search with cross-validation for the best parameters of a GradientBoostingClassifier.

In [ ]:
gbm = GradientBoostingRegressor(
    loss='ls',
    n_estimators=100,
    criterion='friedman_mse',
    min_samples_split=2,
    max_depth=3,
    random_state=0,
    n_iter_no_change=2,
    tol=0.0001,
    )

In [ ]:
param_grid = dict(
    loss=['ls', 'huber'],
    n_estimators=[10, 20, 50, 100, 200, 500, 1000, 2000],
    min_samples_split=[0.1, 0.3, 0.5],
    max_depth=[1,2,3,4,None],
    )

In [ ]:
search = nested_cross_val(gbm, param_grid)

In [ ]:
# reg = GridSearchCV(gbm, param_grid, scoring='neg_mean_squared_error')

# search = reg.fit(X_train, y_train)

# search.best_params_

In [ ]:
results = pd.DataFrame(search.cv_results_)

results.head()

In [ ]:
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['params','mean_test_score', 'std_test_score']]

In [ ]:
results['params'][0]

In [ ]:
results['mean_test_score'].plot(yerr=[results['std_test_score'], results['std_test_score']], subplots=True)

plt.ylabel('Mean test score')

In [ ]:
# let's get the predictions

X_train_preds = search.predict(X_train)
X_test_preds = search.predict(X_test)



In [ ]:
print('Train rmse: ', mean_squared_error(y_train, X_train_preds, squared=False))
print('Test rmse: ', mean_squared_error(y_test, X_test_preds, squared=False))

In [ ]:
print('Train r2: ', r2_score(y_train, X_train_preds))
print('Test r2: ', r2_score(y_test, X_test_preds))

## Light GBM

In [ ]:
# Light GBM

lgbm_param = {
    "num_leaves": [6, 8, 20, 30],
    "max_depth": [2, 4, 6, 8, 10],
    "n_estimators": [50, 100, 200, 500],
    'colsample_bytree': [0.3, 1.0],
}

lgbm = LGBMRegressor(
    learning_rate = 0.1, 
    min_child_weight = 0.4,
    objective='regression', 
    random_state=0)

In [ ]:
search = nested_cross_val(lgbm, lgbm_param)

In [ ]:
# reg = GridSearchCV(lgbm, lgbm_param, scoring='neg_mean_squared_error')

# search = reg.fit(X_train, y_train)

# search.best_params_

In [ ]:
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results['mean_test_score'].plot(yerr=[results['std_test_score'], results['std_test_score']], subplots=True)

plt.ylabel('Mean test score')

In [ ]:
# let's get the predictions
X_train_preds = search.predict(X_train)
X_test_preds = search.predict(X_test)

In [ ]:
print('Train rmse: ', mean_squared_error(y_train, X_train_preds,squared=False))
print('Test rmse: ', mean_squared_error(y_test, X_test_preds,squared=False))

In [ ]:
print('Train r2: ', r2_score(y_train, X_train_preds))
print('Test r2: ', r2_score(y_test, X_test_preds))